In [12]:
import pandas as pd
import numpy as np
import os
import requests
import shutil
from tqdm import tqdm
from facebook_scraper import get_posts
import pythainlp as pythai

In [13]:
post_data = pd.read_csv('showcase_data_cleaned(2022).csv')
print(post_data.columns)
post_data.isna().sum()

Index(['title', 'ID', 'summary', 'title_image', 'images', 'links', 'reason',
       'author', 'fulltext', 'post_url', 'date'],
      dtype='object')


title          0
ID             0
summary        0
title_image    0
images         0
links          0
reason         0
author         0
fulltext       0
post_url       0
date           0
dtype: int64

In [17]:
class Textprocessing:
    def __init__(self) -> None:
        pass
    
    def process_summary(self, x: str) -> list[str]:
        newx = x.split('- ')
        x = [x for x in newx if x != '']
        return newx

    def process_link(self, link: str) -> list[str]:
        newx = link.split('\n')
        newx = [x for x in newx if x != '']
        medium = [x for x in newx if 'medium' in x or 'อ่านรายละเอียดเพิ่มเติม:' in x][0].split(': ')[-1]
        github = [x for x in newx if 'github' in x][0].split(': ')[-1]
        return medium, github

    def process_author(self, link: str) -> list[str]:
        newx = link.split('/')
        name = newx[0]
        bio = '/'.join(newx[1:])
        return name, bio

    def process_name(self, text, root='./posts/2022/'):
        # tokenize
        # tokens = pythai.word_tokenize(text)
        # x = '-'.join(tokens)
        # if len(x) < 20:
        #     x = text
        # elif len(x) > 20:
        #     x = x = ''.join(tokens[:3])
        x = text
            
        naming = x.replace(' ', '-').replace('!', '').replace('?', '').replace(':','')
        path = f"{root}{naming}.md"
        return path

    def make_md(self, data_dict):
        md_format ='''---
date: "{}"
title: "{}"
builder: "{}"
builder_info: "{}"
thumbnail: "{}"
links:
    github: "{}"
    facebook: "{}"
    blog: "{}"
---

![image]({})

{}

### แรงจูงในในการเข้าร่วมโครงการ (จากใบสมัครเข้าร่วมเมื่อ 10 สัปดาห์ที่แล้ว)

> "{}"'''

        return md_format.format(data_dict['date'], data_dict['title'], data_dict['builder'], data_dict['builder_info'], data_dict['thumbnail'][1:], data_dict['links']['github'], data_dict['links']['facebook'], data_dict['links']['blog'], data_dict['thumbnail'][1:], '\n'.join(data_dict['summary']), data_dict['reason'])

processor = Textprocessing()

In [18]:
def extract_all_post_information_from(data) -> list[dict]:
    data = data.copy() # in case data points to the same memory
    posts = []
    for project_index in tqdm(range(data.shape[0])):
        root = './images/2022/'

        info_dict = {'date': "",  "title": "", "builder": "", "builder_info": "", "thumbnail": "", "links": {"github": "", "facebook": "", "blog": ""}, "summary": "", "reason": ""}
        info_dict['date'] = data.iloc[project_index]['date']
        info_dict['title'] = data.iloc[project_index]['title']
        info_dict['builder'], data.iloc[project_index]['builder_info'] = processor.process_author(data.iloc[project_index]['author'])
        # info_dict['thumbnail'] = data.iloc[project_index]['title_image']
        project_id = data.iloc[project_index]['ID'].astype(str)
        info_dict['thumbnail'] = root + project_id + '/' + '01' + '.jpg' # save to static/images/2022/21/01.jpg
        
        info_dict['links']['blog'], info_dict['links']['github'] = processor.process_link(data.iloc[project_index]['links'])
        info_dict['links']['facebook'] = data.iloc[project_index]['post_url']
        info_dict['summary'] = processor.process_summary(data.iloc[project_index]['summary'])
        info_dict['summary'] = new_summary = ['- ' + x.replace("'", '') for x in info_dict['summary'] if x != '']
        info_dict['reason'] = data.iloc[project_index]['reason'].split('แล้ว):')[-1].replace("',", '').replace("'", '').strip()
        
        # strip everyshit
        for key in info_dict.keys():
            if type(info_dict[key]) == str:
                info_dict[key] = info_dict[key].strip()
            elif type(info_dict[key]) == list:
                info_dict[key] = [x.strip() for x in info_dict[key]]
            elif type(info_dict[key]) == dict:
                for key2 in info_dict[key].keys():
                    info_dict[key][key2] = info_dict[key][key2].strip()
        
        posts.append(info_dict)

        # download image and save to static/images in format "/images/2022/21/01.jpg"
        # make folder according to ['ID']
        if os.path.exists(root) == False:
            os.mkdir(root)

        if os.path.exists(root + data.iloc[project_index]['ID'].astype(str)) == False:
            os.mkdir(root + data.iloc[project_index]['ID'].astype(str))

    return posts

posts = extract_all_post_information_from(post_data)

  0%|          | 0/45 [00:00<?, ?it/s]C:\Users\hirun\AppData\Local\Temp\ipykernel_21780\3916541260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_dict['builder'], data.iloc[project_index]['builder_info'] = processor.process_author(data.iloc[project_index]['author'])
C:\Users\hirun\AppData\Local\Temp\ipykernel_21780\3916541260.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_dict['builder'], data.iloc[project_index]['builder_info'] = processor.process_author(data.iloc[project_index]['author'])
C:\Users\hirun\AppData\Local\Temp\ipykernel_21780\3916541260.py:10: SettingWithCopyWarning: 
A value is tryi

In [22]:
def create_allmd_from(data) -> None:
    for post in tqdm(data):
        data = processor.make_md(post)
        # get naming fixed
        fname = processor.process_name(text=post['title'], root="./posts/2022/")
        with open(fname, 'w', encoding='utf8') as f:
            f.write(data)

create_allmd_from(posts)

100%|██████████| 45/45 [00:00<00:00, 1451.46it/s]


In [23]:
# check if the post was generated correctly
print(len(os.listdir('./posts/2022/')))

45
